In [22]:
import cv2
import mediapipe as mp
import time
import math

class HandDetector:
    
    def __init__(self,mode = False,maxHands = 2, complexity = 1, detectionCon = 0.5,trackCon = 0.5):
        
        self.mode = mode
        self.maxHands = maxHands
        self.complexity = complexity
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpHand = mp.solutions.hands
        
        self.Hands = self.mpHand.Hands(self.mode,self.maxHands,self.complexity,
                                       self.detectionCon,self.trackCon)
        
        self.tipIds = [4,8,12,16,20]
        
    def findHands(self,image, draw = True,flipType = True):
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.results = self.Hands.process(imageRGB)
        allHands = []
        h,w,c = image.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness,self.results.multi_hand_landmarks):
                myHand = {}
                mylmList = []
                xList = []
                yList = []
                
                for id,lm in enumerate(handLms.landmark):
                    px,py,pz = int(lm.x*w),int(lm.y*h),int(lm.z*w)
                    mylmList.append([px,py,pz])
                    xList.append(px)
                    yList.append(py)
                    
                xmin,xmax = min(xList),max(xList) 
                ymin,ymax = min(yList),max(yList)
                
                bboxW,bboxH = xmax-xmin,ymax-ymin
                bbox = xmin,ymin,xmax,ymax
                
                cx,cy = bbox[0] + (bboxW // 2),bbox[1] + (bboxH // 2)
                
                myHand["lmList"] = mylmList
                myHand["bbox"] = bbox
                myHand["center"] = (cx,cy)
                
                if flipType:
                    if handType.classification[0].label == "Right":
                        myHand["type"] = "Left"
                    else:
                        myHand["type"] = "Right"
                        
                else:
                    
                    myHand["type"] = handType.classification[0].label
                    
                allHands.append(myHand)
                
                if draw:
                    
                    self.mpDraw.draw_landmarks(image,handLms,self.mpHand.HAND_CONNECTIONS)
                    
                    cv2.rectangle(image,(bbox[0]-20,bbox[1] - 20),(bbox[2]+20,bbox[3] + 20),
                                 (255,0,255),2)
                    
                    cv2.putText(image,myHand["type"],(bbox[0]-30,bbox[1] -30),cv2.FONT_HERSHEY_PLAIN,
                               2,(255,0,255),2)
                    
        if draw:
            return allHands,image
        
        else:
            return allHands
        
        
    def fingersUp(self,myHand,flipType=True):
            
        myHandType = myHand["type"]
        mylmList = myHand["lmList"]
        fingers = []
        if flipType :
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
        else:
            if myHandType == "Right":

                if mylmList[self.tipIds[0]][0] < mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            else:

                if mylmList[self.tipIds[0]][0] > mylmList[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                    
            

        for id in range(1,5):
            if mylmList[self.tipIds[id]][1] < mylmList[self.tipIds[id] - 2][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers
        
        
    def findDistance(self,p1,p2,image = None):
            
        x1,y1 = p1
        x2,y2 = p2
        cx,cy =( x1 + x2 ) // 2,(y1 + y2) // 2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)

        if image is not None:

            cv2.circle(image, (x1,y1),15,(255,0,255),cv2.FILLED)
            cv2.circle(image, (x2,y2),15,(255,0,255),cv2.FILLED)
            cv2.line(image, (x1,y1),(x2,y2),(255,0,255),3)
            cv2.circle(image, (cx,cy),15,(255,0,255),cv2.FILLED)
            return length,info,image

        return length,info

In [42]:
import cv2
import numpy as np
import time

pTime = 0
def fps():
    global pTime
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps


cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

hand_detector = HandDetector(maxHands=2,detectionCon=0.8)

path = "/Users/gokhanersoz/Desktop/Hepsi/OpenCV/PROJECTS/Resources/mediapipeHands.png"
image = cv2.imread(path)
image = cv2.resize(image,(350,250))

startDist = None
scale = 0
cx,cy = 500,500

while True:
    
    conn,frame = cap.read()
    frame = cv2.flip(frame,1)
    
    allHands,frame = hand_detector.findHands(frame,draw = True,flipType = False)
    

    if len(allHands) == 2:

        hands1 = allHands[0]
        lmList1 = hands1["lmList"]
        bbox1 = hands1["bbox"]
        center1 = hands1["center"]
        type1 = hands1["type"]

        hands2 = allHands[1]
        lmList2 = hands2["lmList"]
        bbox2 = hands2["bbox"]
        center2 = hands2["center"]
        type2 = hands2["type"]

        fingers1 = hand_detector.fingersUp(hands1,flipType=False)
        fingers2 = hand_detector.fingersUp(hands2,flipType=False)

        cv2.putText(frame , f"fingers1 : {fingers1}",(20,300),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
        cv2.putText(frame , f"fingers2 : {fingers2}",(20,350),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)            

        if fingers1 == [1,1,0,0,0] and fingers2 == [1,1,0,0,0]:
            cv2.putText(frame , f"Mode : Zoom Gesture",(20,150),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)

            if startDist is None:

                #length4,info4,frame = hand_detector.findDistance(lmList1[4][:-1],lmList2[4][:-1],frame)
                length8,info8,frame = hand_detector.findDistance(lmList1[8][:-1],lmList2[8][:-1],frame)

                # Örnek olarak başlangıç uzaklığı 30 olsun
                startDist = length8

            length8,info8,frame = hand_detector.findDistance(lmList1[8][:-1],lmList2[8][:-1],frame)

            # Sonra biz bunu yeni uzunluk 50 olarak genişlettik
            # Ama uzama olarak 50 - 30 = 20 olarak elimize geçiyor !!!
            scale = int((length8 - startDist) // 2)
            
            # uzunluğun ortası resmin olmasını istediğimiz konum !! 
            cx,cy = info8[4:]
            #print(scale)

    # 2 eli görmedin tekrardan hesaplama yapmak zorunda !!!
    else:
        startDist = None

    try:
        h1,w1,_ = image.shape
        print("Before" , h1 + scale,w1 + scale)
        newH,newW = ((h1 + scale)//2)*2 , ((w1 + scale) // 2)*2
        # tekti çifte çeviriyoruz !!!
        print("After" , newH, newW)
        imageResize = cv2.resize(image,(newW,newH))
        frame[cy - newH // 2: cy + newH // 2,cx - newW // 2:cx + newW // 2] = imageResize
        
    except:
        pass

    
    fps_ = fps()
    cv2.putText(frame , f"Zoom : {int(scale)}",(20,200),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
    cv2.putText(frame , f"FPS : {int(fps_)}",(20,70),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),2)
    cv2.imshow("Frame",frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
        
cap.release()
cv2.destroyAllWindows()

Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350
After 250 350
Before 250 350

Before 255 355
After 254 354
Before 254 354
After 254 354
Before 250 350
After 250 350
Before 247 347
After 246 346
Before 242 342
After 242 342
Before 240 340
After 240 340
Before 240 340
After 240 340
Before 243 343
After 242 342
Before 247 347
After 246 346
Before 258 358
After 258 358
Before 278 378
After 278 378
Before 306 406
After 306 406
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437
After 336 436
Before 337 437